In [ ]:
from skimage.util import random_noise
import numpy as np
from skimage.metrics import mean_squared_error, peak_signal_noise_ratio

In [27]:
def MSE(image1,image2):
  mse = np.mean(np.square(image1.astype(float) - image2.astype(float)))
  
  return mse

def PSNR(image1, image2, peak=255):
  mse = MSE(image1,image2)

  psnr = 10*np.log10(peak**2/mse)

  return psnr

In [25]:
def create_gaussian_noise(img,mean=0,var=0.01):

    # Converting pixel values from 0-255 to 0-1 float
    img = img/255

    # Initializing the result (noisy) image
    result = img.copy()

    # Adding gaussian noise to the image
    gauss = np.random.normal(mean, var**0.5, img.shape)
    result = result + gauss
    result = np.clip(result, 0, 1)

    # Converting the result back to uint8
    result = np.uint8(result*255)

    return result

In [9]:
from numba import jit

@jit(nopython=True, cache=True)
def findAllNeighbors(padImg,small_window,big_window,h,w):
    smallWidth = small_window//2
    bigWidth = big_window//2

    neighbors = np.zeros((padImg.shape[0],padImg.shape[1],small_window,small_window))

    for i in range(bigWidth,bigWidth + h):
        for j in range(bigWidth,bigWidth + w):   
            neighbors[i,j] = padImg[(i - smallWidth):(i + smallWidth + 1) , (j - smallWidth):(j + smallWidth + 1)]
    
    return neighbors

@jit(nopython=True, cache=True)
def evaluateNorm(pixelWindow, neighborWindow, Nw):
    Ip_Numerator,Z = 0,0

    for i in range(neighborWindow.shape[0]):
      for j in range(neighborWindow.shape[1]):
        q_window = neighborWindow[i,j]

        q_x,q_y = q_window.shape[0]//2,q_window.shape[1]//2

        Iq = q_window[q_x, q_y]

        w = np.exp(-1*((np.sum((pixelWindow - q_window)**2))/Nw))

        Ip_Numerator = Ip_Numerator + (w*Iq)
        Z = Z + w

    return Ip_Numerator/Z

In [37]:
class NLMeans():

  def solve(self,img,h=30,small_window=7,big_window=21):
    padImg = np.pad(img,big_window//2,mode='reflect')

    return self.NLM(padImg,img,h,small_window,big_window)

  @staticmethod
  @jit(nopython = True, cache = True)
  def NLM(padImg, img, h, small_window, big_window):
    Nw = (h**2)*(small_window**2)

    h,w = img.shape

    result = np.zeros(img.shape)

    bigWidth = big_window//2
    smallWidth = small_window//2

    neighbors = findAllNeighbors(padImg, small_window, big_window, h, w) 

    for i in range(bigWidth, bigWidth + h):
        for j in range(bigWidth, bigWidth + w):
            pixelWindow = neighbors[i,j]

            neighborWindow = neighbors[(i - bigWidth):(i + bigWidth + 1) , (j - bigWidth):(j + bigWidth + 1)]

            Ip = evaluateNorm(pixelWindow, neighborWindow, Nw)

            result[i - bigWidth, j - bigWidth] = max(min(255, Ip), 0)

    return result

In [38]:
image_number = 3 
salt_and_paper_h =   36
gaussian_h =         27

%matplotlib inline
import cv2
import matplotlib.pyplot as plt
from IPython.display import clear_output
 
clear_output(wait=True)
 
 
denoiser = NLMeans()
image = cv2.imread(f"./astronaut.png")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

gaussian_noised = create_gaussian_noise(image.copy())
  

my_gaussian_denoise = denoiser.solve(gaussian_example.copy(),h=gaussian_h)

 
plt.figure(figsize=(20,15))
plt.subplot(1,3,1)
plt.imshow(image,cmap='gray')
plt.title("Original Image")

plt.subplot(1,3,2)
plt.imshow(image,cmap='gray')
plt.title(f"Gaussian Denoise, PSNR:{PSNR(image,my_gaussian_denoise):.2f}")
plt.show()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mNameError: name 'find_all_neighbors' is not defined[0m